In [40]:
import re
import os
import json
import random
import requests
from pathlib import Path

In [2]:
%%bash
pwd

/mnt/c/Users/alex2/Documents/cs474/project/2-web-app-garage1


In [3]:
url = "https://exercisedb.p.rapidapi.com/exercises"

querystring = {"limit":"2000","offset":"0"}

headers = {
	"x-rapidapi-key": "1df69cbc3bmsh6918f272bceaceap139562jsn9c474bf15421",
	"x-rapidapi-host": "exercisedb.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

In [4]:
result = response.json()

In [30]:
result[0:2]

[{'bodyPart': 'waist',
  'equipment': 'body weight',
  'gifUrl': 'https://v2.exercisedb.io/image/rKz9ULsU97jL9d',
  'id': '0001',
  'name': '3/4 sit-up',
  'target': 'abs',
  'secondaryMuscles': ['hip flexors', 'lower back'],
  'instructions': ['Lie flat on your back with your knees bent and feet flat on the ground.',
   'Place your hands behind your head with your elbows pointing outwards.',
   'Engaging your abs, slowly lift your upper body off the ground, curling forward until your torso is at a 45-degree angle.',
   'Pause for a moment at the top, then slowly lower your upper body back down to the starting position.',
   'Repeat for the desired number of repetitions.']},
 {'bodyPart': 'waist',
  'equipment': 'body weight',
  'gifUrl': 'https://v2.exercisedb.io/image/-fgNq3YlnZsJmg',
  'id': '0002',
  'name': '45° side bend',
  'target': 'abs',
  'secondaryMuscles': ['obliques'],
  'instructions': ['Stand with your feet shoulder-width apart and your arms extended straight down by yo

In [7]:
len(result)

1324

In [25]:
unique_category = set([r['bodyPart'] for r in result])

unique_category

{'back',
 'cardio',
 'chest',
 'lower arms',
 'lower legs',
 'neck',
 'shoulders',
 'upper arms',
 'upper legs',
 'waist'}

In [36]:
exercise_by_body_part = {}

for r in result:
    body_part = r['bodyPart']
    if body_part not in exercise_by_body_part:
        exercise_by_body_part[body_part] = []
    exercise_by_body_part[body_part].append(r)
# exercise_by_body_part

In [44]:
selected_exercises = []
for k, v in exercise_by_body_part.items():
    if len(v) <= 10:
        selected_exercises.extend(v)
    else:
        selected_exercises.extend(random.sample(v, 11))

In [47]:
len(selected_exercises)

101

In [48]:
result = selected_exercises

In [49]:
l = list(result[0].keys())

In [50]:
result[0].keys()

dict_keys(['bodyPart', 'equipment', 'gifUrl', 'id', 'name', 'target', 'secondaryMuscles', 'instructions'])

In [51]:
l[0] = 'categories'
l[5] = 'target_muscle'

In [52]:
l

['categories',
 'equipment',
 'gifUrl',
 'id',
 'name',
 'target_muscle',
 'secondaryMuscles',
 'instructions']

In [53]:
%%bash
pwd
ls
cd images
pwd

/mnt/c/Users/alex2/Documents/cs474/project/2-web-app-garage1
LICENSE
README.md
exercise_data.ipynb
exercise_data.json
images
instructions.md
/mnt/c/Users/alex2/Documents/cs474/project/2-web-app-garage1/images


In [54]:
def clean_filename(name):
    name = name.replace(' ', '_')
    return re.sub(r'[^\w\-]', '', name)

In [55]:
dir = r'C:\Users\alex2\Documents\cs474\project\2-web-app-garage1\images'

for r in result:
    gif_url = r['gifUrl']
    gif_name = r['name']
    
    # print((gif_name, gif_url))
    cleaned_filename = clean_filename(gif_name)
    filename = f'{cleaned_filename}.gif'
    path = os.path.join(dir, filename)
    
    response = requests.get(gif_url)
    with open(path, 'wb') as f:
        f.write(response.content)
        print(f'gif saved: {filename}')

gif saved: flag.gif
gif saved: seated_side_crunch_wall.gif
gif saved: negative_crunch.gif
gif saved: assisted_lying_leg_raise_with_throw_down.gif
gif saved: band_v-up.gif
gif saved: barbell_standing_twist.gif
gif saved: assisted_lying_leg_raise_with_lateral_throw_down.gif
gif saved: kneeling_plank_tap_shoulder_male.gif
gif saved: kettlebell_figure_8.gif
gif saved: lean_planche.gif
gif saved: band_vertical_pallof_press.gif
gif saved: roller_hip_stretch.gif
gif saved: barbell_wide_squat.gif
gif saved: band_lying_hip_internal_rotation.gif
gif saved: dumbbell_stiff_leg_deadlift.gif
gif saved: band_squat.gif
gif saved: barbell_narrow_stance_squat.gif
gif saved: exercise_ball_hip_flexor_stretch.gif
gif saved: barbell_squat_jump_step_rear_lunge.gif
gif saved: iron_cross_stretch.gif
gif saved: dumbbell_sumo_pull_through.gif
gif saved: barbell_glute_bridge.gif
gif saved: dumbbell_reverse_grip_incline_bench_one_arm_row.gif
gif saved: cable_pushdown_straight_arm_v_2.gif
gif saved: smith_back_shru

In [56]:
new_result = []

base_dir = Path.cwd()
images_dir = base_dir / 'images'

print(base_dir)
print(images_dir)

for r in result:
    new_dict = {'categories' if k == 'bodyPart' else 'target_muscle' if k == 'target' else 'gif_path' if k == 'gifUrl' else k:v for k, v in r.items()}
    # print(new_dict)

    gif_url = new_dict['gif_path']
    gif_name = new_dict['name']

    cleaned_filename = clean_filename(gif_name)
    filename = f'{cleaned_filename}.gif'

    file_path = images_dir / filename

    relative_path = os.path.relpath(file_path, start=base_dir).replace('\\', '/')
    new_dict['gif_path'] = relative_path 
    
    # print(relative_path)
    # print(new_dict)
    
    new_result.append(new_dict)

C:\Users\alex2\Documents\cs474\project\2-web-app-garage1
C:\Users\alex2\Documents\cs474\project\2-web-app-garage1\images


In [57]:
len(new_result)

101

In [58]:
json_file = base_dir / 'exercise_data.json'

print(json_file)
with open(json_file, 'w', encoding='utf-8') as json_file:
    json.dump(new_result, json_file, indent=4, ensure_ascii=False)

C:\Users\alex2\Documents\cs474\project\2-web-app-garage1\exercise_data.json
